In [79]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import random
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [80]:
#check if guess
def is_guess(model, row):
    if not model.has_index_for(row['question']):
        return True
    for i in range(4):
        if model.has_index_for(row[str(i)]):
            return False
    return True

In [81]:
file = open('sample_6.txt', 'r')
data_unclean = file.readlines()
block_size = 6

for i in range(int(len(data_unclean)/block_size)):
    question = data_unclean[(i*6)].split(".\t")[1].strip()
    c1 = np.array([data_unclean[(i*6)+1].split(".\t")[0].strip(),data_unclean[(i*6)+1].split(".\t")[1].strip()])
    c2 = np.array([data_unclean[(i*6)+2].split(".\t")[0].strip(),data_unclean[(i*6)+2].split(".\t")[1].strip()])
    c3 = np.array([data_unclean[(i*6)+3].split(".\t")[0].strip(),data_unclean[(i*6)+3].split(".\t")[1].strip()])
    c4 = np.array([data_unclean[(i*6)+4].split(".\t")[0].strip(),data_unclean[(i*6)+4].split(".\t")[1].strip()])
    ans_letter = data_unclean[(i*6)+5].strip()
    if ans_letter == 'a':
        ans = c1[1]
    elif ans_letter == 'b':
        ans = c2[1]
    elif ans_letter == 'c':
        ans = c3[1]
    else:
        ans = c4[1]
    pd.DataFrame([question, ans, c1[1], c2[1], c3[1], c4[1]]).T.to_csv(
    'sample_6.csv', sep=',', index=False, header=False, mode='a')

    
df = pd.read_csv("sample_6.csv", header=None)
df.to_csv("sample_6.csv", header=['question', 'answer', '0', '1', '2', '3'], index=False)

In [82]:
#find the closest synonym to the question-word. Use the similarity method from Gensim.
def guess_synonym(model):
    data = pd.read_csv('sample_6.csv')
    output_data = []
    most_similar = 0
    guess_word = ""
    label = ""
    for row in data.to_dict(orient="records"):
        for i in range(4):
            if (is_guess(model, row)):
                random_word_key = random.randint(0,3)
                guess_word = row[str(random_word_key)]
                label = "guess"
                break
            if not model.has_index_for(row[str(i)]):
                continue
            else:
                model_similarity = model.similarity(row['question'], row[str(i)])
            if (model_similarity > most_similar):
                most_similar = model_similarity
                guess_word = row[str(i)]
        if (label != "guess"):
            label = "correct" if (guess_word == row['answer']) else "wrong"

        output_data.append({"question": row['question'], "answer": row['answer'], "guess": guess_word, "label": label})
        most_similar = 0
        guess_word = ""
        label = ""
    return output_data

In [83]:
#write results to csv file
def write_results(model_name, data):
    fieldnames = data[0].keys()
    fieldnames = output_data[0].keys()
    pd.DataFrame(output_data, columns=fieldnames).to_csv(
    f'{model_name}-details-demo.csv', sep=',', index=False)

In [84]:
#write analysis file
def write_analysis(model_name, model, results):
    vocab_size = len(model)
    results_per_label = Counter(element['label'] for element in results)
    c = results_per_label['correct']
    v = 20-results_per_label['guess']
    model_accuracy = c/v
    pd.DataFrame([model_name, vocab_size, c, v, model_accuracy]).T.to_csv(
    'analysis-demo.csv', sep=',', index=False, header=False, mode='a')

In [85]:
#load the pretrained embedding models !!WARNING!! TAKES A WHILE TO RUN
model_names = ["word2vec-google-news-300","glove-twitter-50", "glove-wiki-gigaword-50", "glove-wiki-gigaword-100", "glove-wiki-gigaword-200"]
models = np.array([])
for model_name in model_names:
    models = np.append(models, api.load(model_name))

In [86]:
# store the results, perform the analysis
for i in range(len(model_names)):
    output_data = guess_synonym(models[i])
    write_results(model_names[i], output_data)
    write_analysis(model_names[i], models[i], output_data)